In [ ]:
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# NPL Season 1 (2024) Analysis
The Nepal Premier League (NPL) is a T20 cricket league in Nepal. It was founded by the Cricket Association of Nepal (CAN) in 2024. The inaugural season was held from November 30, 2024 (15 Mangsir) to December 6, 2024 (6 Poush 2081 BS). The league features eight teams based on provinces and cities. The tournament follows a round-robin format, with the top teams advancing to the playoffs.


![Image](https://wicketnepal.com/wp-content/uploads/2024/11/468719762_122127843350455690_7601835368897814823_n.jpg)

There are 2 input files for this analysis. The `NPL matches` file is collected manually, and the `NPL_final` file is scraped from 2 websites ( [espncricinfo](https://espncricinfo.com)  &  [cricket](https://cricket.com) ) and cleaned manually. The `NPL_final` file is a combination of 2 datasets scraped from the commentary section, so some data may not be accurate. For example, both websites have some missing data, and the missing data was collected manually. Additionally, the commentary section of one website may differ from the other. For instance, in Match 1, Inning 1, Over 15.3, Rajesh Pulami is shown as run out on espncricinfo but as stumped on cricket.com. Therefore, there may be some inaccuracies in the data from both websites.

In [ ]:
npl = pd.read_csv('/kaggle/input/nepal-premier-league-2024-ball-by-ball-data/NPL matches.csv')
ball = pd.read_csv('/kaggle/input/nepal-premier-league-2024-ball-by-ball-data/NPL_final.csv')

In [ ]:
print(npl.shape)
print(ball.shape)

The `NPL matches` file contains 32 rows and 27 columns, with each row representing one match. Therefore, 32 matches were played in Season 1.

The `NPL_final` file contains 7,485 rows and 20 columns, with each row representing one ball bowled. In total, 7,485 balls were bowled in Season 1 of the NPL.

In [ ]:
print(npl.info())
print(ball.info())

## Input Files
The only missing datas are from `NPL_final` file with 4 columns having less data.

In [ ]:
npl.head(5)

In [ ]:
ball.head(5)

## Data Cleaning 
Some data cleaning might be needed, but I have not performed it in this notebook because it was already addressed during data collection. However, for deeper or more detailed analysis, additional cleaning could still be necessary.

# Toss Decision

In [ ]:
toss_counts = npl['toss_decision'].value_counts()
plt.figure(figsize=(6,6))
plt.pie(toss_counts, labels=toss_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Toss Decision Distribution')
plt.show()

**65.5%** of the time, the captain chose to bowl first after winning the toss.

# Toss Impact
Does winning the toss increase your chances of winning the match?

In [ ]:
toss_winner_advantage = (npl['toss_winner'] == npl['winner']).mean() * 100
data = pd.DataFrame({
    'Category': ['Matches Won by Toss Winner', 'Matches Lost by Toss Winner'],
    'Percentage': [toss_winner_advantage, 100 - toss_winner_advantage]
})

plt.figure(figsize=(8, 5))
sns.barplot(x='Category', 
    y='Percentage', 
    data=data, 
    palette=['#2ecc71', '#e74c3c']
)
plt.title('Toss Winner Advantage', fontsize=14)
plt.ylabel('Percentage (%)', fontsize=12)
plt.ylim(0, 60)
plt.show()

The graph shows that winning the toss does not necessarily guarantee victory. Teams have won the match only **43%** of the time after winning the toss.

# Chasing Vs Defending
Let's see whether batting or bowling first is advantageous in the inaugural edition of the NPL.

In [ ]:
# Calculate Chasing & Defending Wins
chasing_wins = len(npl[npl['win_by_wickets'] > 0])
defending_wins = len(npl[npl['win_by_runs'] > 0])
super_over_matches = len(npl) - (chasing_wins + defending_wins)

data = pd.DataFrame({
    'Category': ['Chasing Wins', 'Defending Wins', 'Super Over/Tie'],
    'Matches': [chasing_wins, defending_wins, super_over_matches]
})

plt.figure(figsize=(8, 5))
sns.barplot(
    x='Category', 
    y='Matches', 
    data=data, 
    palette=['#2ecc71', '#e74c3c', '#f39c12']
)
plt.title('Match Outcomes: Chasing vs Defending', fontsize=14)
plt.ylabel('Number of Matches', fontsize=12)
plt.show()

The graph above shows that it didn't matter whether a team was batting or bowling first, as 16 times teams won while chasing the game and 15 times teams won while defending the score.

**1 game ended in a Super Over**

# Win Percentage
The winning rate of each team with respect to the number of matches they have played.

In [ ]:
all_teams = pd.concat([npl['team_1'], npl['team_2']])  
matches_played = all_teams.value_counts().reset_index()
matches_played.columns = ['Team', 'Matches']  

wins = npl['winner'].value_counts().reset_index() 
wins.columns = ['Team', 'Wins'] 

# Step 3: Merge matches played and wins, then calculate win percentage
win_percent = pd.merge(matches_played, wins, on='Team', how='left')  
win_percent['Wins'] = win_percent['Wins'].fillna(0) 
win_percent['Win Percentage'] = (win_percent['Wins'] / win_percent['Matches']) * 100

# Sort by win percentage for better visualization
win_percent = win_percent.sort_values('Win Percentage', ascending=True)  

num_teams = len(win_percent)
colors = plt.cm.cividis([i / num_teams for i in range(num_teams)])  

plt.figure(figsize=(10, 6))  
plt.barh(
    win_percent['Team'], 
    win_percent['Win Percentage'], 
    color=colors 
)

plt.xlabel('Win Percentage', fontsize=12)
plt.ylabel('Team', fontsize=12)
plt.title('Team Win Percentages', fontsize=16)

# Adding percentage values on the bars
for index, value in enumerate(win_percent['Win Percentage']):
    plt.text(value, index, f'{value:.1f}%', va='center')
plt.tight_layout()
plt.show()

`Sudurpaschim Royals` has the highest win percentage in the tournament, with 77.8% (7 wins in 9 matches).

# Team Discipline (Extras Conceded)
Extra runs conceded by each team in the tournament 

In [ ]:
team_extras = {}
for team in npl['team_1'].unique():
    team_extras[team] = npl[npl['team_1'] == team]['team_1_runs_from_extra'].sum() + \
                        npl[npl['team_2'] == team]['team_2_runs_from_extra'].sum()
    
extras_df = pd.DataFrame(list(team_extras.items()), columns=['Team', 'Extras']).sort_values('Extras', ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x='Extras', y='Team', data=extras_df, palette='mako')
plt.title('Total Extras Conceded by Teams')
plt.show()

total_extras = npl['team_1_runs_from_extra'].sum() + npl['team_2_runs_from_extra'].sum()
print(f"Total Extras in Tournament: {total_extras}")

`Janakpur Bolts` is the team that has conceded the most extras, with a total of 106 runs in extras, averaging **10.6** per match.

# Top 5 Players with the most man of the match awards

In [ ]:
player_counts = npl['player_of_the_match'].value_counts().head(5)

plt.figure(figsize=(12,7))
sns.barplot(x=player_counts.values, y=player_counts.index, palette='viridis')

plt.title('Top 5 Players of the Match')
plt.xlabel('Number of Awards')
plt.ylabel("Players")

plt.gca().xaxis.set_major_formatter(plt.FormatStrFormatter('%d'))
plt.show()

# Batsmen Section
## Top Batsmen of the Tournament
Viewing the top 10 run scorers from Season 1

In [ ]:
runs = ball.groupby('batsman')['batsman_runs'].sum().sort_values(ascending=False)

top_batsmen = runs[:10].reset_index()  
top_batsmen.columns = ['Batsman', 'Runs']  

fig = px.bar(
    top_batsmen,
    x='Batsman',
    y='Runs',
    hover_data=['Batsman', 'Runs'],
    color='Runs',
    title='Top 10 Batsmen in NPL Season 1'
)
fig.show(renderer='iframe')

## Runs Scored by a Batsman Against Each Team
The dropdown button displays the runs scored by a batsman against each team. Only batsmen who have scored more than 130 runs are included in the dropdown.

In [ ]:
from ipywidgets import widgets, HBox, VBox, interactive_output

# Calculate total runs per batsman
total_runs = ball.groupby('batsman')['batsman_runs'].sum().reset_index()
qualified_batsmen = total_runs[total_runs['batsman_runs'] > 130]['batsman'].tolist()

# Dropdown for batsman selection
batsman_dropdown = widgets.Dropdown(
    options=qualified_batsmen,
    description='Batsman:',
    disabled=False
)

# Function to update the graph based on dropdown selections
def update_plot(batsman_name):
    mask = ball['batsman'] == batsman_name
    filtered_data = ball[mask]
    runs_by_team = filtered_data.groupby('bowling_team')['batsman_runs'].sum().sort_values()
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=runs_by_team.values,
        y=runs_by_team.index,
        orientation='h',
        name='Runs Scored',
        marker_color='#FF4545'
    ))
    
    fig.update_layout(
        title=f'Runs Scored by {batsman_name} Against Each Team',
        xaxis_title='Runs Scored',
        yaxis_title='Bowling Team',
        showlegend=False,
    )
    fig.show(renderer='iframe')

# Link dropdowns to the plot
output = interactive_output(update_plot, {'batsman_name': batsman_dropdown})

# Display widgets and graph
display(VBox([HBox([batsman_dropdown]), output]))

Rohit Paudel scored 95 runs against *Pokhara Avengers* and 80 runs against *Chitwan Rhinos*, among others. You can replace his name with any other player to retrieve their data.

## Batsmen with the most 6's in the tournament

In [ ]:
mask = ball['batsman_runs'] == 6
new_ball = ball[mask]

dots = new_ball.groupby('batsman').size().sort_values(ascending=False).head(10)
top_batsmen = dots[:10].reset_index()  # Reset index to make it a DataFrame
top_batsmen.columns = ['Batsman', 'Runs']  # Rename columns for clarity

fig = px.bar(
    top_batsmen,
    x='Batsman',
    y='Runs',
    hover_data=['Batsman', 'Runs'],
    color='Runs',
    title="Top 10 Batsmen with the most 6's in NPL Season 1"
)
fig.show(renderer='iframe')

## Batsmen with the most 4's in the tournament

In [ ]:
mask = ball['batsman_runs'] == 4
new_ball = ball[mask]

dots = new_ball.groupby('batsman').size().sort_values(ascending=False).head(10)
top_batsmen = dots[:10].reset_index()  # Reset index to make it a DataFrame
top_batsmen.columns = ['Batsman', 'Runs']

fig = px.bar(
    top_batsmen,
    x='Batsman',
    y='Runs',
    hover_data=['Batsman', 'Runs'],
    color='Runs',
    title="Top 10 Batsmen with the most 4's in NPL Season 1"
)
fig.show(renderer='iframe')

## Comparing Batsmen with the Most Dot Balls and Their Strike Rates
Below is a bar graph showing the batsmen with the most dot balls played and their respective strike rates in the tournament. This illustrates whether the batsmen have compensated for their dot balls with hitting, as reflected in their strike rates.

In [ ]:
filtered_ball = ball[(ball["wide_runs"] == 0) & (ball["noball_runs"] == 0)]

# Aggregating batting statistics
batting_stats = filtered_ball.groupby("batsman").agg(
    total_runs=pd.NamedAgg(column="batsman_runs", aggfunc="sum"),
    balls_faced=pd.NamedAgg(column="ball_over", aggfunc="count"),
).reset_index()

batting_stats["strike_rate"] = (batting_stats["total_runs"] / batting_stats["balls_faced"]) * 100
batting_stats = batting_stats.fillna(0).sort_values(by="total_runs", ascending=False)

dot_balls = ball[ball["batsman_runs"] == 0].groupby("batsman").size().reset_index(name="dot_balls")
dot_balls = dot_balls.merge(batting_stats[["batsman", "strike_rate"]], on="batsman", how="left")

top_dot_ball_players = dot_balls.sort_values(by="dot_balls", ascending=False).head(10)

# MELTING DATA TO LONG FORMAT FOR GROUPED BAR CHART
melted_data = top_dot_ball_players.melt(id_vars=["batsman"], value_vars=["dot_balls", "strike_rate"],
                                        var_name="Metric", value_name="Value")

# Custom colors for the bars
first_bar = '#32628d'  # Color for Dot Balls
second_bar = '#cde01d'  # Color for Strike Rate

# Creating horizontal grouped bar chart with small gap between same Metric and large gap between different Metrics
plt.figure(figsize=(12, 6))
sns.barplot(x="Value", y="batsman", hue="Metric", data=melted_data, 
            palette=[first_bar, second_bar], orient="h", width=0.8)

# Adjusting space between the groups manually
plt.subplots_adjust(left=0.1, right=0.9, top=1.4, bottom=0.1)

# Labels and title
plt.xlabel("Value")
plt.ylabel("Batsman")
plt.title("Top 10 Players with the most Dot Balls and their Strike Rate")

plt.show()
top_dot_ball_players[["batsman", "dot_balls", "strike_rate"]]

### Analysing Dot Balls and Strike Rates

From the graph above, we can see that `Binod Bhandari` has played the most dot balls, but his strike rate is above 110, which is considered good for a T20 match. 

On the other hand, `Dev Khanal` has the lowest strike rate among the batsmen with the most dot balls, at 82.25. However, it's important to note that this analysis does not take into account the match situation, the batsman's batting position, or the required runs, all of which might contribute to a lower strike rate. Therefore, this metric is not intended to target any individual player but to highlight which players have compensated for their dot balls the most.

`Saif Zaib` has the highest strike rate among the batsmen with the most dot balls played, meaning he has compensated the most for his dot balls.


# Bowlers Section
## Top Bowlers of the Tournament
Viewing the top 10 wicket takers form season 1

In [ ]:
mask = ball[~ball['dismissal_kind'].isin(['Runout', 'Retired Hurt'])]

wickets = mask.groupby('bowler')['dismissal_kind'].count().reset_index(name='Wickets')
top_bowlers = wickets.sort_values(by='Wickets', ascending=False).head(10)
top_bowlers.columns = ['Bowler', 'Wickets']

fig = px.bar(
    top_bowlers, 
    x='Wickets', 
    y='Bowler', 
    hover_data=['Bowler'], 
    color='Wickets',
    title='Top 10 Bowlers in NPL 2024',
    orientation='h'
)

fig.show(renderer='iframe')

## Bowlers with the Most Dot Balls
These are the bowlers who delivered the highest number of dot balls in the tournament.

In [ ]:
mask = (ball['batsman_runs'] == 0) & (ball['noball_runs'] == 0) & (ball['wide_runs'] == 0)
new_ball = ball[mask]

dots = new_ball.groupby('bowler').size().sort_values(ascending=False).head(10)

# Convert to DataFrame
top_bowlers = dots.reset_index()
top_bowlers.columns = ['Bowler', 'Balls']

plt.figure(figsize=(10, 6))
colors = plt.cm.viridis(top_bowlers['Balls'] / max(top_bowlers['Balls'])) 
bars = plt.bar(top_bowlers['Bowler'], top_bowlers['Balls'], color=colors)

# Add numbers above bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{int(height)}', 
             ha='center', va='bottom', fontsize=10)

plt.title('Top 10 Bowlers with the Most Dot Balls in NPL Season 1', fontsize=14)
plt.xlabel('Bowler', fontsize=12)
plt.ylabel('Dot Balls', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

`Lalit Rajbanshi` is by far the best bowler in the tournament, leading in both wickets and dot balls while maintaining an economy rate under 6.

# Distribution of Shot Directions in the Tournament
This pie chart visualizes the distribution of different shot directions used by players during the tournament. It highlights the frequency of each shot direction, offering insights into the most preferred areas for scoring runs. The chart provides a clear view of the players' shot selection tendencies throughout the games.

In [ ]:
shot_direction_counts = ball['shot_direction'].value_counts()

plt.figure(figsize=(8, 6))
sns.set(style="whitegrid")

plt.pie(shot_direction_counts, labels=shot_direction_counts.index, autopct='%1.1f%%', startangle=90, colors=sns.color_palette("Set2", len(shot_direction_counts)))
plt.title('Distribution of Shot Directions')
plt.show()

## Shot Direction Analysis for Individual Players
This shows the preferred shot direction of each player. You can select a player's name from the dropdown menu below to view their favorite region for scoring runs. To keep the list manageable, only batsmen with more than 130 runs are included in the dropdown.

In [ ]:
total_runs = ball.groupby('batsman')['batsman_runs'].sum().reset_index()
qualified_batsmen = total_runs[total_runs['batsman_runs'] > 130]['batsman'].tolist()

# Dropdown for batsman selection
batsman_dropdown = widgets.Dropdown(
    options=qualified_batsmen,
    description='Batsman:',
    disabled=False
)

# Function to update the graph based on dropdown selections
def update_shot_direction(batsman_name):
    mask = ball['batsman'] == batsman_name
    filtered_data = ball[mask]
    direction = filtered_data.groupby('shot_direction')['batsman_runs'].sum().sort_values(ascending=True)
    
    fig = px.bar(
        x=direction.values, 
        y=direction.index, 
        orientation='h', 
        labels={'x': 'Runs Scored', 'y': 'Shot Direction'},
        title=f'Shot Direction Preferred by {batsman_name}',
        color_discrete_sequence=['#16C47F']
    )
    
    fig.show(renderer='iframe')

# Link dropdowns to the plot
output = interactive_output(update_shot_direction, {'batsman_name': batsman_dropdown})
display(VBox([HBox([batsman_dropdown]), output]))

# Top 5 Best Pairs in the Tournament
The top 5 pairs in the tournament were determined by calculating the total partnership runs scored by each duo across all the games they played together.

In [ ]:
# Create sorted player pairs to treat (A,B) and (B,A) as the same partnership
ball_sorted = ball.copy()
# Sort batsman and non_striker names alphabetically to create consistent pairs
ball_sorted[['player1', 'player2']] = ball_sorted.apply(
    lambda row: sorted([row['batsman'], row['non_striker']]), 
    axis=1, 
    result_type='expand'
)

# Group by the sorted pairs and sum runs
partnerships = (
    ball_sorted.groupby(['player1', 'player2'])['batsman_runs']
    .sum()
    .reset_index()
    .sort_values('batsman_runs', ascending=False)
    .head(5)
)

# Partnership label
partnerships['partnership'] = partnerships['player1'] + ' & ' + partnerships['player2']

fig = px.bar(
    partnerships,
    x='partnership',  
    y='batsman_runs',
    title='Top 5 Batting Partnerships (Overall)',
    labels={'batsman_runs': 'Partnership Runs'},
    color='partnership',
)
fig.show(renderer='iframe')

`Andries Gous` and `Dinseh Kharel` are the best duo in the tournament, with 327 runs scored between them throughout the competition.

# Phase-wise Bowling Performance
Analyzing the wickets taken by bowlers according to the different phases of the game.

In [ ]:
ball['ball_over'] = pd.to_numeric(ball['ball_over'], errors='coerce')
ball['phase'] = pd.cut(ball['ball_over'],
                      bins = [0, 6, 16, 20],
                      labels = ['Powerplay','Middle','Death'])

mask = ~ball['dismissal_kind'].isin(['Runout', 'Retired Hurt'])
filtered_ball = ball[mask]

# when doing groupby use reset_index() to convert it into structured dataframe
phase_wk = filtered_ball.groupby(['bowler','phase'], observed=False)['dismissal_kind'].count().reset_index()

total_wk = phase_wk.groupby('bowler')['dismissal_kind'].sum().reset_index()
total_wk.rename(columns={'dismissal_kind':'total_wk'}, inplace=True)

# Sorting accordin to total wickets
sorted_bowlers = total_wk[total_wk['total_wk'] > 6].sort_values(by='total_wk', ascending=False)['bowler']

phase_wk = phase_wk.merge(total_wk, on='bowler')
phase_wk = phase_wk[phase_wk['bowler'].isin(sorted_bowlers)]

final= phase_wk.pivot(index='bowler', columns='phase', values='dismissal_kind').fillna(0)
final = final.reindex(index=sorted_bowlers) 

plt.figure(figsize=(50, 40))
sns.heatmap(final, cmap='viridis', annot=True, annot_kws={'size': 20})
plt.title("Wickets Taken by Bowlers in Different Phases (Minimum 7 Wickets)", fontsize=26, pad=30)
plt.xlabel("Match Phase", fontsize=30, labelpad=30)
plt.ylabel("Bowlers", fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.show()

From the above heatmap, we can see that `Rijan Dhakal` and `Sohail Tanvir` have the most wickets in **Powerplay Overs** with 9 wickets each, while `William Bosisto`, `Gerhard Erasmus`, and `Mohammad Mohsin` dominate the **Middle Overs** with 8 wickets each. Lastly, `Kishore Mahato` has the most wickets in **Death Overs** with 10 wickets.

# Phase-wise Batting Performance
This section highlights the **phase-wise batting performance** of all players in **Season 1**. The phases are categorized as follows:  
- **Powerplay (0-6 Overs)**  
- **Middle Overs (6-16 Overs)**  
- **Death Overs (16-20 Overs)**  

You can view the **runs scored** and **strike rate** of each player during every phase of the game.  

- `Dipendra Singh Airee` and `Ravi Bopara` emerge as the most lethal players in the **death overs**, among players with more than 50 runs and a strike rate exceeding **200**.  
- `James Neesham` dominates the **middle overs**, scoring just over 200 runs with an impressive strike rate of **201**. 

In [ ]:
# Preprocessing
ball['ball_over'] = pd.to_numeric(ball['ball_over'], errors='coerce')
ball['phase'] = pd.cut(ball['ball_over'],
                      bins=[0, 6, 16, 20],
                      labels=['Powerplay', 'Middle', 'Death'],
                      right=False)

phase_sr = ball[ball['wide_runs'] == 0].groupby(['batsman', 'phase'], observed=False)['batsman_runs'].agg(['sum', 'count'])
phase_sr['sr'] = (phase_sr['sum'] / phase_sr['count']) * 100
phase_sr = phase_sr.reset_index().sort_values('sr', ascending=False)
fig = px.scatter(phase_sr, x='sum', y='sr', color='phase', hover_data=['batsman'],
                  title='Phase-wise Batting Performance', labels={'sum': 'Total Runs', 'sr': 'Strike Rate'})
fig.show(renderer='iframe')

## Team-wise Batting Performance by Phase
In this section, you can view the phase-wise batting performance for each team of your choice by selecting from the dropdown menu.

In [ ]:
# Preprocessing
ball['ball_over'] = pd.to_numeric(ball['ball_over'], errors='coerce')
ball['phase'] = pd.cut(
    ball['ball_over'],
    bins=[0, 6, 16, 20],
    labels=['Powerplay', 'Middle', 'Death'],
    right=False
)

# Get unique teams for dropdown
teams = ball['batting_team'].unique().tolist()

# Creating empty figure object with dropdown
fig = go.Figure()

for team in teams:
    # Filter data for the team
    team_data = ball[ball['batting_team'] == team]
    
    # Calculate strike rate for the team
    phase_sr = team_data[team_data['wide_runs'] == 0].groupby(['batsman', 'phase'], observed=False)['batsman_runs'].agg(['sum', 'count'])
    phase_sr['sr'] = (phase_sr['sum'] / phase_sr['count']) * 100
    phase_sr = phase_sr.reset_index()
    
    # Add traces for each phase (hidden by default)
    for phase in ['Powerplay', 'Middle', 'Death']:
        phase_df = phase_sr[phase_sr['phase'] == phase]
        fig.add_trace(
            go.Scatter(
                x=phase_df['sum'],
                y=phase_df['sr'],
                mode='markers',
                name=f"{phase}",
                visible=False,  # Start with all traces hidden
                marker=dict(
                    color=dict(Powerplay='#1f77b4', Middle='#ff7f0e', Death='#2ca02c')[phase],
                    size=8
                ),
                hovertext=phase_df['batsman'],
                hoverinfo='text+x+y'
            )
        )

# Create dropdown buttons
dropdown_buttons = []
for i, team in enumerate(teams):
    visibility = [False] * len(teams) * 3
    visibility[i*3:(i+1)*3] = [True, True, True]
    dropdown_buttons.append(
        dict(
            label=team,
            method='update',
            args=[{'visible': visibility},
                  {'title': f'Phase-wise Performance: {team}'}]
        )
    )

# Update layout
fig.update_layout(
    title='Phase-wise Batting Performance - Select Team',
    xaxis_title='Total Runs',
    yaxis_title='Strike Rate',
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction='down',
        pad={'r': 10, 't': 10},
        showactive=True,
        x=0.1,
        xanchor='left',
        y=1.15,
        yanchor='top'
    )],
    showlegend=True
)

fig.data[0].visible = True
fig.data[1].visible = True
fig.data[2].visible = True
fig.show(renderer='iframe')

# Match Progression 
This graph shows the match progression throughout both innings of a match. It displays the wickets, with overs on the X-axis and runs on the Y-axis, while also highlighting a critical partnership in the inning. 

**NOTE: You can select any match from the dropdown menu to view the match progression.**

In [ ]:
match_dropdown = widgets.Dropdown(
    options=sorted(ball['match_id'].unique()), 
    description='Match ID:',
    disabled=False
)

def update_match_graph(match_id):
    match_data = ball[ball['match_id'] == match_id].sort_values('ball_over')
    match_data['runs'] = match_data.groupby('inning')['total_runs'].cumsum()

    # Get final scores for each inning
    inning_scores = match_data.groupby('inning')['runs'].max().to_dict()

    # Extract batting teams
    batting_teams = match_data[['inning', 'batting_team']].drop_duplicates()
    custom_colors = {
        batting_teams.iloc[0]['batting_team']: "#1f77b4",
        batting_teams.iloc[1]['batting_team']: "#ff7f0e"
    }

    # Score labels for legend
    score_labels = {}
    for _, row in batting_teams.iterrows():
        team = row['batting_team']
        inning = row['inning']
        score_labels[team] = f"{team} ({inning_scores.get(inning, 0)})"

    # Map innings to batting teams
    match_data['batting_team'] = match_data['inning'].map(dict(zip(batting_teams['inning'], batting_teams['batting_team'])))

    # Line Graph
    fig = px.line(match_data, x='ball_over', y='runs', color='batting_team',
                  title=f'Match {match_id} Run Progression',
                  color_discrete_map=custom_colors)

    # Update legend names with scores
    for trace in fig.data:
        original_name = trace.name
        trace.name = score_labels.get(original_name, original_name)

    # Add wickets
    wickets = match_data[match_data['ball_result'].isin(['W', '1W', '2W'])]
    fig.add_trace(go.Scatter(
        x=wickets['ball_over'],
        y=wickets['runs'],
        mode='markers',
        marker=dict(size=8, color='#B82132', symbol="hexagon", line=dict(width=0.8, color='Black')),
        name="Wicket"
    ))

    fig.update_layout(
        xaxis_title="Over",
        yaxis_title="Total Runs",
        legend_title="Batting Team (Total Runs)",
        xaxis=dict(range=[0, 20])
    )
    fig.show(renderer='iframe')

# Link dropdown to the plot
output = interactive_output(update_match_graph, {'match_id': match_dropdown})
display(VBox([match_dropdown, output]))